In [7]:
import gzip
import os
from netCDF4 import Dataset
from glob import glob
import numpy as np
import seaborn as sns
import tarfile


import zipfile
from glob import glob
import os
from netCDF4 import Dataset
import pandas as pd
import geopandas as gpd # GeoPandas(지오판다스)
from shapely.geometry import Polygon, LineString, Point
path = os.getcwd()

import rasterio
from rasterio.plot import show
from rasterio.transform import from_origin
import matplotlib.pyplot as plt

import numpy as np

import requests
import time

from geocube.api.core import make_geocube
import seaborn as sns

In [23]:
# emission 117번 시나리오 없음
emiss_s_list = glob(os.path.join("d:","Emission","*Jul.tar.gz",))
emiss_s_num = [os.path.split(path)[-1].split("_")[1] for path in emiss_s_list]
emiss_info_df = pd.DataFrame()
emiss_info_df.loc[:,'path'] = emiss_s_list
emiss_info_df.loc[:,'s_num'] = emiss_s_num

concentration_s_list = glob(os.path.join("d:","Concentration","*.tar.gz",))
concentration_s_num = [os.path.split(path)[-1].split("_")[1] for path in concentration_s_list]
concentration_info_df = pd.DataFrame()
concentration_info_df.loc[:,'path'] = concentration_s_list
concentration_info_df.loc[:,'s_num'] = concentration_s_num

all_conc_emis_pathinfo = pd.merge(emiss_info_df,concentration_info_df,how='left',on='s_num', suffixes=['_emission','_concentration'])

In [26]:
all_conc_emis_pathinfo

,path_emission,s_num,path_concentration
0,d:Emission\RSM_105_Apr_Jul.tar.gz,105,d:Concentration\RSM_105_ACONC.tar.gz
1,d:Emission\RSM_106_Apr_Jul.tar.gz,106,d:Concentration\RSM_106_ACONC.tar.gz
2,d:Emission\RSM_107_Apr_Jul.tar.gz,107,d:Concentration\RSM_107_ACONC.tar.gz
3,d:Emission\RSM_108_Apr_Jul.tar.gz,108,d:Concentration\RSM_108_ACONC.tar.gz
4,d:Emission\RSM_109_Apr_Jul.tar.gz,109,d:Concentration\RSM_109_ACONC.tar.gz
...,...,...,...
113,d:Emission\RSM_100_Apr_Jul.tar.gz,100,d:Concentration\RSM_100_ACONC.tar.gz
114,d:Emission\RSM_101_Apr_Jul.tar.gz,101,d:Concentration\RSM_101_ACONC.tar.gz
115,d:Emission\RSM_102_Apr_Jul.tar.gz,102,d:Concentration\RSM_102_ACONC.tar.gz
116,d:Emission\RSM_103_Apr_Jul.tar.gz,103,d:Concentration\RSM_103_ACONC.tar.gz


In [ ]:
path = '/home/workspace'
data_path = os.path.join(path,'cmaqProjectdata')
proj_path = os.path.join(path,'repos','cmaqProject')

In [ ]:
# 기상측정망, 대기질측정망 메타정보 정리
em_meta = pd.read_csv(glob(os.path.join(path,"시간단위_기상자료개방포털","rawdata","meta","*.csv",))[0],encoding='cp949')
geo_point = []
for i in range(len(em_meta)):
    x,y = em_meta.loc[i,'경도'], em_meta.loc[i,'위도']
    geo_point.append(Point(x,y))
    

em_meta['geometry'] = geo_point

em_point_gdf = gpd.GeoDataFrame(em_meta, geometry= 'geometry')
em_point_gdf.crs = {'init':'epsg:4326'}

url = 'http://apis.data.go.kr/B552584/MsrstnInfoInqireSvc/getMsrstnList'
key = "k5wXUhoJHwee1cncQCBmm81YbQ+exttb0vdJcyF5GuGJn0mbGBNNL/ER2VfkrJMlExfc+FZjPeRuOM2bvgDYyQ=="

# params ={'serviceKey' : key, 'addr': '서울', 'stationName': '종로구', 'pageNo': 1, 'numOfRows': 10,'returnType': 'json'}
params ={'serviceKey' : key, 'pageNo': 1, 'numOfRows': 640,'returnType': 'json'}
response = requests.get(url, params=params, verify=False)

data_dic = response.json()
site_info = pd.DataFrame(data_dic['response']['body']['items'])

point_list = []
for i in range(len(site_info)):
    try:
        point_list.append(Point(float(site_info.dmY[i]), float(site_info.dmX[i])))
    except:
        point_list.append(None)

site_info.loc[:,'geometry'] = point_list
site_info_dropna = site_info.loc[~site_info.isna().dmX.values]

site_info_dropna = gpd.GeoDataFrame(site_info_dropna, geometry = 'geometry')
site_info_dropna.crs = em_point_gdf.crs

In [ ]:
# 대기질자료 불러오기
airpolutant_monitoring_2013_list = glob(os.path.join(path,"시간단위대기질자료","rawdata","extract","2013*",))
airpolutant_monitoring_2013_list
data_list = []
for i in airpolutant_monitoring_2013_list:
    temp_data = pd.read_excel(i)
    data_list.append(temp_data)
air_pol_2013 = pd.concat(data_list)
air_pol_2013.info()

#기상자료 불러오기
em_dataset_path_list = glob(os.path.join(path,"시간단위_기상자료개방포털","rawdata","extract","*.csv",))
em_dataset_list = [pd.read_csv(path, encoding='cp949') for path in em_dataset_path_list]
em_df = pd.concat(em_dataset_list, axis = 0)
em_df.info()
datetime_airpol = [datetime.datetime(int(date[:4]),int(date[4:6]),int(date[6:8]),int(date[8:] if date[8:] !='24' else '0')) for date in air_pol_2013.측정일시.values.astype(str)]
air_pol_2013.loc[:,'datetime'] = datetime_airpol
em_df.loc[:,'datetime'] = pd.to_datetime(em_df.일시)

In [38]:
# lcc프로젝션 기본 레스터 제작
projout = '+proj=lcc +lat_1=30 +lat_2=60 +lon_1=126 +lat_0=38 +lon_0=126 +ellps=GRS80 +units=m'

# out_grid = make_geocube(vector_data=test_data, measurements=["습도(%)"], geom=geom,resolution=(9000, 9000), fill=0, output_crs=projout) #for most crs negative comes first in resolution
x_m = list(range(-180000,-180000 + 9000 * 68, 9000))
y_m = list(range(-585000,-585000 + 9000 * 83, 9000))

print(len(x_m), len(y_m))

grid_points = []
for x_i in x_m:
    for y_i in y_m:
        grid_points.append(Point(x_i,y_i))

grid_data = pd.DataFrame(grid_points, columns=['geometry'])

grid_data = gpd.GeoDataFrame(grid_data, geometry='geometry')
grid_data.crs = em_point_gdf.to_crs(projout).crs
grid_data.loc[:,'x_m'] = grid_data.geometry.x
grid_data.loc[:,'y_m'] = grid_data.geometry.y
grid_data.loc[:,'value'] = 0

out_grid = make_geocube(vector_data=grid_data, measurements=["value"],resolution=(9000, 9000), fill=0, output_crs=projout) #for most crs negative comes first in resolution
out_grid["value"].rio.to_raster("base_lcc.tif")
# 레스터 좌표변환
# 5181로 변환하면 좌표들 약간 틀어짐
# lcc로 처리 해야함
from rasterio.warp import calculate_default_transform, reproject, Resampling
# dst_crs = 'epsg: 5181'
dst_crs = '+proj=lcc +lat_1=30 +lat_2=60 +lon_1=126 +lat_0=38 +lon_0=126 +ellps=GRS80 +units=m'

with rasterio.open(os.path.join(path,"base_lcc.tif")) as src:

    transform, width, height = calculate_default_transform(
                    src.crs, 
                    dst_crs, 
                    src.width, 
                    src.height, 
                    *src.bounds)
   
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    with rasterio.open(os.path.join(path,"base_lcc_lcc.tif"), 'w', **kwargs) as dst:
            
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)

68 83


c:\Users\Go\anaconda3\envs\tensorgpu\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [39]:
# pm10 합산해서 계산하는 수식 및 chem 리스트
pm_10_cal = "(1.0*ASO4J[1])+(1.0*ASO4I[1])+(1.0*ANH4J[1])+(1.0*ANH4I[1])+(1.0*ANO3J[1])+(1.0*ANO3I[1])+(1.0*AALKJ[1])+(1.0*AXYL1J[1])+(1.0*AXYL2J[1])+(1.0*AXYL3J[1])+(1.0*ATOL1J[1])+(1.0*ATOL2J[1])+(1.0*ATOL3J[1])+(1.0*ABNZ1J[1])+(1.0*ABNZ2J[1])+(1.0*ABNZ3J[1])+(1.0*ATRP1J[1])+(1.0*ATRP2J[1])+(1.0*AISO1J[1])+(1.0*AISO2J[1])+(1.0*AISO3J[1])+(1.0*ASQTJ[1])+(1.0*AORGCJ[1])+(1.0*AORGPAJ[1])+(1.0*AORGPAI[1])+(1.0*AECJ[1])+(1.0*AECI[1])+(1.0*A25J[1])+(1.0*ANAJ[1])+(1.0*ACLJ[1])+(1.0*ACLI[1])+(1.0*AOLGAJ[1])+(1.0*AOLGBJ[1])+(1.0*ACORS[1])+(1.0*ASOIL[1])+(1.0*ANAK[1])+(1.0*ACLK[1])+(1.0*ASO4K[1])+(1.0*ANH4K[1])+(1.0*ANO3K[1])"
pm_10_chem_list = [chem[5:-4] for chem in pm_10_cal.split("+")]

In [40]:
import tensorflow as tf
from tqdm import tqdm

In [41]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [42]:
projout = '+proj=lcc +lat_1=30 +lat_2=60 +lon_1=126 +lat_0=38 +lon_0=126 +ellps=GRS80 +units=m'
col_list = ['기온(°C)', '강수량(mm)', '풍속(m/s)', '풍향(16방위)', '습도(%)', '증기압(hPa)', '이슬점온도(°C)', '현지기압(hPa)', '해면기압(hPa)', '일조(hr)', '일사(MJ/m2)', '적설(cm)', '3시간신적설(cm)', '전운량(10분위)', '중하층운량(10분위)',]


In [49]:
for i in range(len(all_conc_emis_pathinfo))[1:2]:
    path_emis = all_conc_emis_pathinfo.path_emission.values[i]
    path_conc = all_conc_emis_pathinfo.path_concentration.values[i]
    s_num = all_conc_emis_pathinfo.s_num[i]
    
    print(path_emis, path_conc)
    tar = tarfile.open(path_conc,"r:gz")
    tar.extractall(os.path.join("e:","dataset_record","Concentration",))
    tar.close()

    tar = tarfile.open(path_emis,"r:gz")
    tar.extractall(os.path.join("e:","dataset_record","Emission",))
    tar.close()


    # 날짜별 smoke, cmaq데이터셋 경로 정리
    emis_list = glob(os.path.join("e:","dataset_record","Emission",f"RSM_{s_num}","*"))         
    concentration_list = glob(os.path.join("e:","dataset_record","Concentration",f"RSM_{s_num}","*"))           

    emis_day_list = [os.path.split(path)[-1].split('.')[-5] for path in emis_list]
    concentration_day_list = [os.path.split(path)[-1].split('.')[-2] for path in concentration_list]

    
    concentration_day_list_datetime = []
    for date in concentration_day_list:

        start_date = datetime.date(int(date[:4]), 1, 1) + datetime.timedelta(-1)
        d_day = int(date[4:])
        target_date = start_date + datetime.timedelta(d_day)

        concentration_day_list_datetime.append(target_date)

    emis_day_list_datetime = []
    for date in emis_day_list:
        target_date = datetime.date(int(date[:4]), int(date[4:6]), int(date[6:]))
        emis_day_list_datetime.append(target_date)

    emis_info_df = pd.DataFrame()
    emis_info_df.loc[:,['date']] = emis_day_list_datetime
    emis_info_df.loc[:,['path']] = emis_list

    concentration_info_df = pd.DataFrame()
    concentration_info_df.loc[:,['date']] = concentration_day_list_datetime
    concentration_info_df.loc[:,['path']] = concentration_list
    concentration_info_df = concentration_info_df.sort_values('date')
    concentration_info_df.index = range(len(concentration_info_df))

    con_emis_info_merged_df = pd.merge(concentration_info_df, emis_info_df, how='left', on='date', suffixes=['_concentration', '_emission'])
        
    con_emis_info_merged_df_dropna = con_emis_info_merged_df.loc[~con_emis_info_merged_df.path_emission.isna(),]
    con_emis_info_merged_df_dropna.index = range(len(con_emis_info_merged_df_dropna))

    writer_image = tf.io.TFRecordWriter(os.path.join("e:","dataset_record",f"dataset_{s_num}.tfr"))
    print(os.path.join("e:","dataset_record",f"dataset_{s_num}.tfr"))

    # for i in range(len(con_emis_info_merged_df_dropna)):
    for i in range(len(con_emis_info_merged_df_dropna))[:1]:
    
        today = con_emis_info_merged_df_dropna.loc[i].date
        print(today)
        # 해당날짜의 데이터셋 불러오기
        with Dataset(con_emis_info_merged_df_dropna.loc[i].path_concentration, 'r') as concen_data, Dataset(con_emis_info_merged_df_dropna.loc[i].path_emission, 'r') as emis_data:

            # concen_data = Dataset(con_emis_info_merged_df_dropna.loc[i].path_concentration, 'r')
            # emis_data = Dataset(con_emis_info_merged_df_dropna.loc[i].path_emission, 'r')

            # 해당날짜의 관련변수 합으로 pm10 CMAQ array형성
            pm10_chemlist = [np.array(concen_data.variables[chem]) for chem in pm_10_chem_list]
            day_concen_pm10 = np.sum(pm10_chemlist, axis=0)
            day_concen_pm10 = day_concen_pm10.reshape(day_concen_pm10.shape[0],day_concen_pm10.shape[2],day_concen_pm10.shape[3],-1)

            # 해당날짜의 smoke 모든변수 array 형성, 연직방향으로는 맨 밑레이어만 사용
            smoke_chem_list = list(set(emis_data.variables.keys()) - set(['TFLAG']))
            smoke_chem_arr_list = [np.array(emis_data.variables[chem])[:,0:1,:,:].reshape(25,82,67,-1) for chem in smoke_chem_list]
            day_smoke_allval = np.concatenate(smoke_chem_arr_list, axis=3)
            day_smoke_allval = day_smoke_allval[:24]

            print('CMAQ_PM10 데이터셋: ',day_concen_pm10.shape, 'SMOKE_모든변수 데이터셋: ',day_smoke_allval.shape)

            # for hour_ind in range(24):
            for hour_ind in range(4):
                datetime_ind = datetime.datetime(today.year, today.month, today.day, hour_ind)
                print(str(datetime_ind))

                air_pol_datetime_ind_df = air_pol_2013.loc[air_pol_2013.datetime == datetime_ind]
                air_pol_datetime_ind_df.index = range(len(air_pol_datetime_ind_df))

                air_pol_datetime_ind_df_geoinfo = pd.merge(air_pol_datetime_ind_df,site_info_dropna.loc[:,['stationName', 'geometry']],how='left',left_on='측정소명',right_on='stationName')
                air_pol_datetime_ind_df_geoinfo = air_pol_datetime_ind_df_geoinfo.loc[~air_pol_datetime_ind_df_geoinfo.stationName.isna()]
                air_pol_datetime_ind_df_geoinfo.index = range(len(air_pol_datetime_ind_df_geoinfo))
                air_pol_datetime_ind_df_geoinfo = gpd.GeoDataFrame(air_pol_datetime_ind_df_geoinfo, geometry='geometry')
                air_pol_datetime_ind_df_geoinfo = air_pol_datetime_ind_df_geoinfo.fillna(0)
                air_pol_datetime_ind_df_geoinfo = air_pol_datetime_ind_df_geoinfo.loc[~(air_pol_datetime_ind_df_geoinfo.PM10 == -999)]  # 값이 -999찍혀있을때가 있음 기기오작동으로보고 제거
                air_pol_datetime_ind_df_geoinfo.index = range(len(air_pol_datetime_ind_df_geoinfo))

                # out_grid = make_geocube(vector_data=test_data, measurements=["습도(%)"], geom=geom,resolution=(9000, 9000), fill=0, output_crs=projout) #for most crs negative comes first in resolution
                out_grid = make_geocube(vector_data=air_pol_datetime_ind_df_geoinfo, measurements=["PM10"],resolution=(9000, 9000), fill=0, output_crs=projout) #for most crs negative comes first in resolution
                out_grid["PM10"].rio.to_raster("rasterized_air_pol.tif")


                dst_crs = projout
                with rasterio.open(os.path.join(path,"rasterized_air_pol.tif")) as src:
                    # print(src.width, src.height, *src.bounds,)
                    transform, width, height = calculate_default_transform(
                                    src.crs, 
                                    dst_crs, 
                                    src.width, 
                                    src.height, 
                                    *src.bounds)
                
                    kwargs = src.meta.copy()
                    kwargs.update({
                        'crs': dst_crs,
                        'transform': transform,
                        'width': width,
                        'height': height
                    })

                    with rasterio.open(os.path.join(path,"rasterized_air_pol_lcc.tif"), 'w', **kwargs) as dst:
                            
                            for i in range(1, src.count + 1):
                                reproject(
                                    source=rasterio.band(src, i),
                                    destination=rasterio.band(dst, i),
                                    src_transform=src.transform,
                                    src_crs=src.crs,
                                    dst_transform=transform,
                                    dst_crs=dst_crs,
                                    resampling=Resampling.nearest)

            

                with rasterio.open(os.path.join(path,"base_lcc_lcc.tif")) as src, rasterio.open(os.path.join(path,"rasterized_air_pol_lcc.tif")) as src2:
                    raster_list = [src, src2]
                    # print(src.read(1).shape, src2.read(1).shape,)
                    mosaic, output = merge(raster_list)

                    output_meta = src2.meta.copy()
                    output_meta.update(
                        {"driver": "GTiff",
                            "height": mosaic.shape[1],
                            "width": mosaic.shape[2],
                            "transform": output,
                        }
                    )

                    with rasterio.open(os.path.join(path,"rasterized_air_pol_lcc_final.tif"), "w", **output_meta) as m:
                        m.write(mosaic)

                with rasterio.open(os.path.join(path,"rasterized_air_pol_lcc_final.tif")) as src:
                    air_pol_pm10_arr = src.read(1)
                    air_pol_pm10_arr = air_pol_pm10_arr.reshape(1,82,67,1)
                    # print(air_pol_pm10_arr.shape)
                em_df_ind_df = em_df.loc[em_df.datetime == datetime_ind]
                em_df_ind_df.index = range(len(em_df_ind_df))
                em_df_ind_df_geoinfo = pd.merge(em_df_ind_df,em_point_gdf.loc[:,['지점', 'geometry']],how='left',on = '지점')

                em_df_ind_df_geoinfo = em_df_ind_df_geoinfo.loc[~em_df_ind_df_geoinfo.geometry.isna()]
                em_df_ind_df_geoinfo.index = range(len(em_df_ind_df_geoinfo))
                em_df_ind_df_geoinfo = gpd.GeoDataFrame(em_df_ind_df_geoinfo, geometry='geometry')
                em_df_ind_df_geoinfo = em_df_ind_df_geoinfo.fillna(0)


                # out_grid = make_geocube(vector_data=test_data, measurements=["습도(%)"], geom=geom,resolution=(9000, 9000), fill=0, output_crs=projout) #for most crs negative comes first in resolution

                em_arr_list = []
                for col_name in col_list:
                    
                    out_grid = make_geocube(vector_data=em_df_ind_df_geoinfo, measurements=[col_name],resolution=(9000, 9000), fill=0, output_crs=projout) #for most crs negative comes first in resolution
                    out_grid[col_name].rio.to_raster("rasterized_em.tif")

                    dst_crs = projout
                    with rasterio.open(os.path.join(path,"rasterized_em.tif")) as src:
                        # print(src.count)
                        transform, width, height = calculate_default_transform(
                                        src.crs, 
                                        dst_crs, 
                                        src.width, 
                                        src.height, 
                                        *src.bounds)
                    
                        kwargs = src.meta.copy()
                        kwargs.update({
                            'crs': dst_crs,
                            'transform': transform,
                            'width': width,
                            'height': height
                        })

                        with rasterio.open(os.path.join(path,"rasterized_em_lcc.tif"), 'w', **kwargs) as dst:
                                
                                for i in range(1, src.count + 1):
                                    reproject(
                                        source=rasterio.band(src, i),
                                        destination=rasterio.band(dst, i),
                                        src_transform=src.transform,
                                        src_crs=src.crs,
                                        dst_transform=transform,
                                        dst_crs=dst_crs,
                                        resampling=Resampling.nearest)



                    

                    with rasterio.open(os.path.join(path,"base_lcc_lcc.tif")) as src, rasterio.open(os.path.join(path,"rasterized_em_lcc.tif")) as src2:
                        raster_list = [src, src2]
                        # print(src.read(1).shape, src2.read(1).shape,)
                        mosaic, output = merge(raster_list)

                        output_meta = src2.meta.copy()
                        output_meta.update(
                            {"driver": "GTiff",
                                "height": mosaic.shape[1],
                                "width": mosaic.shape[2],
                                "transform": output,
                            }
                        )

                        with rasterio.open(os.path.join(path,"rasterized_em_lcc_final.tif"), "w", **output_meta) as m:
                            m.write(mosaic)

                    with rasterio.open(os.path.join(path,"rasterized_em_lcc_final.tif")) as src:
                    
                        em_arr = src.read(1)
                        em_arr = em_arr.reshape(1,82,67,1)
                        em_arr_list.append(em_arr)

                em_arr_map = np.concatenate(em_arr_list,axis=3)

                weather_t = em_arr_map[:1].tobytes()
                air_q_t = air_pol_pm10_arr[:1].tobytes()
                cmaq_t = day_concen_pm10[hour_ind:hour_ind+1].tobytes()
                smoke_t = day_smoke_allval[hour_ind:hour_ind+1].tobytes()

                example = tf.train.Example(
                    features=tf.train.Features(
                        feature={
                            'CMAQ_t': _bytes_feature(cmaq_t),
                            'SMOKE_t': _bytes_feature(smoke_t),
                            'air_quality_monitoring_t': _bytes_feature(air_q_t),   
                            'weather_monitoring_t': _bytes_feature(weather_t),
                            'year': _bytes_feature(np.array(today.year - 2000).reshape(1,-1).tobytes()), 
                            'month': _bytes_feature(np.array(today.month).reshape(1,-1).tobytes()), 
                            'day': _bytes_feature(np.array(today.day).reshape(1,-1).tobytes()), 
                            'hour': _bytes_feature(np.array(hour_ind).reshape(1,-1).tobytes()), 

                        }
                        )
                    )
                writer_image.write(example.SerializeToString())
        
    writer_image.close()

    shutil.rmtree(os.path.join("e:","dataset_record","Concentration",f"RSM_{s_num}"))
    shutil.rmtree(os.path.join("e:","dataset_record","Emission",f"RSM_{s_num}"))

    

d:Emission\RSM_106_Apr_Jul.tar.gz d:Concentration\RSM_106_ACONC.tar.gz
e:dataset_record\dataset_106.tfr
2013-03-22
CMAQ_PM10 데이터셋:  (24, 82, 67, 1) SMOKE_모든변수 데이터셋:  (24, 82, 67, 45)
2013-03-22 00:00:00
2013-03-22 01:00:00
2013-03-22 02:00:00
2013-03-22 03:00:00


In [ ]:
all_conc_emis_pathinfo

In [ ]:
for i in range(len(all_conc_emis_pathinfo))[1:]:
    path_emis = all_conc_emis_pathinfo.path_emission.values[i]
    path_conc = all_conc_emis_pathinfo.path_concentration.values[i]
    s_num = all_conc_emis_pathinfo.s_num[i]
    
    if s_num != 106:
        print(path_emis, path_conc)
        tar = tarfile.open(path_conc,"r:gz")
        tar.extractall(os.path.join("e:","dataset_record","Concentration",))
        tar.close()

        tar = tarfile.open(path_emis,"r:gz")
        tar.extractall(os.path.join("e:","dataset_record","Emission",))
        tar.close()


    # 날짜별 smoke, cmaq데이터셋 경로 정리
    emis_list = glob(os.path.join("e:","dataset_record","Emission",f"RSM_{s_num}","*"))         
    concentration_list = glob(os.path.join("e:","dataset_record","Concentration",f"RSM_{s_num}","*"))           

    emis_day_list = [os.path.split(path)[-1].split('.')[-5] for path in emis_list]
    concentration_day_list = [os.path.split(path)[-1].split('.')[-2] for path in concentration_list]

    
    concentration_day_list_datetime = []
    for date in concentration_day_list:

        start_date = datetime.date(int(date[:4]), 1, 1) + datetime.timedelta(-1)
        d_day = int(date[4:])
        target_date = start_date + datetime.timedelta(d_day)

        concentration_day_list_datetime.append(target_date)

    emis_day_list_datetime = []
    for date in emis_day_list:
        target_date = datetime.date(int(date[:4]), int(date[4:6]), int(date[6:]))
        emis_day_list_datetime.append(target_date)

    emis_info_df = pd.DataFrame()
    emis_info_df.loc[:,['date']] = emis_day_list_datetime
    emis_info_df.loc[:,['path']] = emis_list

    concentration_info_df = pd.DataFrame()
    concentration_info_df.loc[:,['date']] = concentration_day_list_datetime
    concentration_info_df.loc[:,['path']] = concentration_list
    concentration_info_df = concentration_info_df.sort_values('date')
    concentration_info_df.index = range(len(concentration_info_df))

    con_emis_info_merged_df = pd.merge(concentration_info_df, emis_info_df, how='left', on='date', suffixes=['_concentration', '_emission'])
        
    con_emis_info_merged_df_dropna = con_emis_info_merged_df.loc[~con_emis_info_merged_df.path_emission.isna(),]
    con_emis_info_merged_df_dropna.index = range(len(con_emis_info_merged_df_dropna))

    writer_image = tf.io.TFRecordWriter(os.path.join("e:","dataset_record",f"dataset_{s_num}.tfr"))
    print(os.path.join("e:","dataset_record",f"dataset_{s_num}.tfr"))

    for i in range(len(con_emis_info_merged_df_dropna)):
    # for i in range(len(con_emis_info_merged_df_dropna))[:1]:
    
        today = con_emis_info_merged_df_dropna.loc[i].date
        print(today)
        # 해당날짜의 데이터셋 불러오기
        with Dataset(con_emis_info_merged_df_dropna.loc[i].path_concentration, 'r') as concen_data, Dataset(con_emis_info_merged_df_dropna.loc[i].path_emission, 'r') as emis_data:

            # concen_data = Dataset(con_emis_info_merged_df_dropna.loc[i].path_concentration, 'r')
            # emis_data = Dataset(con_emis_info_merged_df_dropna.loc[i].path_emission, 'r')

            # 해당날짜의 관련변수 합으로 pm10 CMAQ array형성
            pm10_chemlist = [np.array(concen_data.variables[chem]) for chem in pm_10_chem_list]
            day_concen_pm10 = np.sum(pm10_chemlist, axis=0)
            day_concen_pm10 = day_concen_pm10.reshape(day_concen_pm10.shape[0],day_concen_pm10.shape[2],day_concen_pm10.shape[3],-1)

            # 해당날짜의 smoke 모든변수 array 형성, 연직방향으로는 맨 밑레이어만 사용
            smoke_chem_list = list(set(emis_data.variables.keys()) - set(['TFLAG']))
            smoke_chem_arr_list = [np.array(emis_data.variables[chem])[:,0:1,:,:].reshape(25,82,67,-1) for chem in smoke_chem_list]
            day_smoke_allval = np.concatenate(smoke_chem_arr_list, axis=3)
            day_smoke_allval = day_smoke_allval[:24]

            print('CMAQ_PM10 데이터셋: ',day_concen_pm10.shape, 'SMOKE_모든변수 데이터셋: ',day_smoke_allval.shape)

            for hour_ind in range(24):
            # for hour_ind in range(4):
                datetime_ind = datetime.datetime(today.year, today.month, today.day, hour_ind)
                print(str(datetime_ind))

                air_pol_datetime_ind_df = air_pol_2013.loc[air_pol_2013.datetime == datetime_ind]
                air_pol_datetime_ind_df.index = range(len(air_pol_datetime_ind_df))

                air_pol_datetime_ind_df_geoinfo = pd.merge(air_pol_datetime_ind_df,site_info_dropna.loc[:,['stationName', 'geometry']],how='left',left_on='측정소명',right_on='stationName')
                air_pol_datetime_ind_df_geoinfo = air_pol_datetime_ind_df_geoinfo.loc[~air_pol_datetime_ind_df_geoinfo.stationName.isna()]
                air_pol_datetime_ind_df_geoinfo.index = range(len(air_pol_datetime_ind_df_geoinfo))
                air_pol_datetime_ind_df_geoinfo = gpd.GeoDataFrame(air_pol_datetime_ind_df_geoinfo, geometry='geometry')
                air_pol_datetime_ind_df_geoinfo = air_pol_datetime_ind_df_geoinfo.fillna(0)
                air_pol_datetime_ind_df_geoinfo = air_pol_datetime_ind_df_geoinfo.loc[~(air_pol_datetime_ind_df_geoinfo.PM10 == -999)]  # 값이 -999찍혀있을때가 있음 기기오작동으로보고 제거
                air_pol_datetime_ind_df_geoinfo.index = range(len(air_pol_datetime_ind_df_geoinfo))

                # out_grid = make_geocube(vector_data=test_data, measurements=["습도(%)"], geom=geom,resolution=(9000, 9000), fill=0, output_crs=projout) #for most crs negative comes first in resolution
                out_grid = make_geocube(vector_data=air_pol_datetime_ind_df_geoinfo, measurements=["PM10"],resolution=(9000, 9000), fill=0, output_crs=projout) #for most crs negative comes first in resolution
                out_grid["PM10"].rio.to_raster("rasterized_air_pol.tif")


                dst_crs = projout
                with rasterio.open(os.path.join(path,"rasterized_air_pol.tif")) as src:
                    # print(src.width, src.height, *src.bounds,)
                    transform, width, height = calculate_default_transform(
                                    src.crs, 
                                    dst_crs, 
                                    src.width, 
                                    src.height, 
                                    *src.bounds)
                
                    kwargs = src.meta.copy()
                    kwargs.update({
                        'crs': dst_crs,
                        'transform': transform,
                        'width': width,
                        'height': height
                    })

                    with rasterio.open(os.path.join(path,"rasterized_air_pol_lcc.tif"), 'w', **kwargs) as dst:
                            
                            for i in range(1, src.count + 1):
                                reproject(
                                    source=rasterio.band(src, i),
                                    destination=rasterio.band(dst, i),
                                    src_transform=src.transform,
                                    src_crs=src.crs,
                                    dst_transform=transform,
                                    dst_crs=dst_crs,
                                    resampling=Resampling.nearest)

            

                with rasterio.open(os.path.join(path,"base_lcc_lcc.tif")) as src, rasterio.open(os.path.join(path,"rasterized_air_pol_lcc.tif")) as src2:
                    raster_list = [src, src2]
                    # print(src.read(1).shape, src2.read(1).shape,)
                    mosaic, output = merge(raster_list)

                    output_meta = src2.meta.copy()
                    output_meta.update(
                        {"driver": "GTiff",
                            "height": mosaic.shape[1],
                            "width": mosaic.shape[2],
                            "transform": output,
                        }
                    )

                    with rasterio.open(os.path.join(path,"rasterized_air_pol_lcc_final.tif"), "w", **output_meta) as m:
                        m.write(mosaic)

                with rasterio.open(os.path.join(path,"rasterized_air_pol_lcc_final.tif")) as src:
                    air_pol_pm10_arr = src.read(1)
                    air_pol_pm10_arr = air_pol_pm10_arr.reshape(1,82,67,1)
                    # print(air_pol_pm10_arr.shape)
                em_df_ind_df = em_df.loc[em_df.datetime == datetime_ind]
                em_df_ind_df.index = range(len(em_df_ind_df))
                em_df_ind_df_geoinfo = pd.merge(em_df_ind_df,em_point_gdf.loc[:,['지점', 'geometry']],how='left',on = '지점')

                em_df_ind_df_geoinfo = em_df_ind_df_geoinfo.loc[~em_df_ind_df_geoinfo.geometry.isna()]
                em_df_ind_df_geoinfo.index = range(len(em_df_ind_df_geoinfo))
                em_df_ind_df_geoinfo = gpd.GeoDataFrame(em_df_ind_df_geoinfo, geometry='geometry')
                em_df_ind_df_geoinfo = em_df_ind_df_geoinfo.fillna(0)


                # out_grid = make_geocube(vector_data=test_data, measurements=["습도(%)"], geom=geom,resolution=(9000, 9000), fill=0, output_crs=projout) #for most crs negative comes first in resolution

                em_arr_list = []
                for col_name in col_list:
                    
                    out_grid = make_geocube(vector_data=em_df_ind_df_geoinfo, measurements=[col_name],resolution=(9000, 9000), fill=0, output_crs=projout) #for most crs negative comes first in resolution
                    out_grid[col_name].rio.to_raster("rasterized_em.tif")

                    dst_crs = projout
                    with rasterio.open(os.path.join(path,"rasterized_em.tif")) as src:
                        # print(src.count)
                        transform, width, height = calculate_default_transform(
                                        src.crs, 
                                        dst_crs, 
                                        src.width, 
                                        src.height, 
                                        *src.bounds)
                    
                        kwargs = src.meta.copy()
                        kwargs.update({
                            'crs': dst_crs,
                            'transform': transform,
                            'width': width,
                            'height': height
                        })

                        with rasterio.open(os.path.join(path,"rasterized_em_lcc.tif"), 'w', **kwargs) as dst:
                                
                                for i in range(1, src.count + 1):
                                    reproject(
                                        source=rasterio.band(src, i),
                                        destination=rasterio.band(dst, i),
                                        src_transform=src.transform,
                                        src_crs=src.crs,
                                        dst_transform=transform,
                                        dst_crs=dst_crs,
                                        resampling=Resampling.nearest)



                    

                    with rasterio.open(os.path.join(path,"base_lcc_lcc.tif")) as src, rasterio.open(os.path.join(path,"rasterized_em_lcc.tif")) as src2:
                        raster_list = [src, src2]
                        # print(src.read(1).shape, src2.read(1).shape,)
                        mosaic, output = merge(raster_list)

                        output_meta = src2.meta.copy()
                        output_meta.update(
                            {"driver": "GTiff",
                                "height": mosaic.shape[1],
                                "width": mosaic.shape[2],
                                "transform": output,
                            }
                        )

                        with rasterio.open(os.path.join(path,"rasterized_em_lcc_final.tif"), "w", **output_meta) as m:
                            m.write(mosaic)

                    with rasterio.open(os.path.join(path,"rasterized_em_lcc_final.tif")) as src:
                    
                        em_arr = src.read(1)
                        em_arr = em_arr.reshape(1,82,67,1)
                        em_arr_list.append(em_arr)

                em_arr_map = np.concatenate(em_arr_list,axis=3)

                weather_t = em_arr_map[:1].tobytes()
                air_q_t = air_pol_pm10_arr[:1].tobytes()
                cmaq_t = day_concen_pm10[hour_ind:hour_ind+1].tobytes()
                smoke_t = day_smoke_allval[hour_ind:hour_ind+1].tobytes()

                example = tf.train.Example(
                    features=tf.train.Features(
                        feature={
                            'CMAQ_t': _bytes_feature(cmaq_t),
                            'SMOKE_t': _bytes_feature(smoke_t),
                            'air_quality_monitoring_t': _bytes_feature(air_q_t),   
                            'weather_monitoring_t': _bytes_feature(weather_t),
                            'year': _bytes_feature(np.array(today.year - 2000).reshape(1,-1).tobytes()), 
                            'month': _bytes_feature(np.array(today.month).reshape(1,-1).tobytes()), 
                            'day': _bytes_feature(np.array(today.day).reshape(1,-1).tobytes()), 
                            'hour': _bytes_feature(np.array(hour_ind).reshape(1,-1).tobytes()), 

                        }
                        )
                    )
                writer_image.write(example.SerializeToString())
        
    writer_image.close()

    shutil.rmtree(os.path.join("e:","dataset_record","Concentration",f"RSM_{s_num}"))
    shutil.rmtree(os.path.join("e:","dataset_record","Emission",f"RSM_{s_num}"))

    

* 최신

In [87]:
path = '/home/workspace'
data_path = os.path.join(path,'cmaqProjectdata')
record_path = os.path.join(data_path,'dataset_record')
proj_path = os.path.join(path,'repos','cmaqProject')
band_depth = 15 # 기상자료에 들어가는 밴드뎁스 상황에따라 바꿔주기
col_list = ['기온(°C)', '강수량(mm)', '풍속(m/s)', '풍향(16방위)', '습도(%)', '증기압(hPa)', '이슬점온도(°C)', '현지기압(hPa)', '해면기압(hPa)', '일조(hr)', '일사(MJ/m2)', '적설(cm)', '3시간신적설(cm)', '전운량(10분위)', '중하층운량(10분위)',]
pm_10_cal = "(1.0*ASO4J[1])+(1.0*ASO4I[1])+(1.0*ANH4J[1])+(1.0*ANH4I[1])+(1.0*ANO3J[1])+(1.0*ANO3I[1])+(1.0*AALKJ[1])+(1.0*AXYL1J[1])+(1.0*AXYL2J[1])+(1.0*AXYL3J[1])+(1.0*ATOL1J[1])+(1.0*ATOL2J[1])+(1.0*ATOL3J[1])+(1.0*ABNZ1J[1])+(1.0*ABNZ2J[1])+(1.0*ABNZ3J[1])+(1.0*ATRP1J[1])+(1.0*ATRP2J[1])+(1.0*AISO1J[1])+(1.0*AISO2J[1])+(1.0*AISO3J[1])+(1.0*ASQTJ[1])+(1.0*AORGCJ[1])+(1.0*AORGPAJ[1])+(1.0*AORGPAI[1])+(1.0*AECJ[1])+(1.0*AECI[1])+(1.0*A25J[1])+(1.0*ANAJ[1])+(1.0*ACLJ[1])+(1.0*ACLI[1])+(1.0*AOLGAJ[1])+(1.0*AOLGBJ[1])+(1.0*ACORS[1])+(1.0*ASOIL[1])+(1.0*ANAK[1])+(1.0*ACLK[1])+(1.0*ASO4K[1])+(1.0*ANH4K[1])+(1.0*ANO3K[1])"
pm_10_chem_list = [chem[5:-4] for chem in pm_10_cal.split("+")]


projout = '+proj=lcc +lat_1=30 +lat_2=60 +lon_1=126 +lat_0=38 +lon_0=126 +ellps=GRS80 +units=m'


def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [80]:
concentration_path_list = np.array(glob(os.path.join(data_path, "extract","concentration", "*")))
emission_path_list = np.array(glob(os.path.join(data_path, "extract","emission", "*")))

senario_list = list(set([path.split("/")[-1] for path in concentration_path_list]) & set([path.split("/")[-1] for path in emission_path_list]))

In [81]:
# 기상측정망, 대기질측정망 메타정보 정리
em_meta = pd.read_csv(glob(os.path.join(data_path,"시간단위_기상자료개방포털","rawdata","meta","*.csv",))[0],encoding='cp949')
geo_point = []
for i in range(len(em_meta)):
    x,y = em_meta.loc[i,'경도'], em_meta.loc[i,'위도']
    geo_point.append(Point(x,y))
    

em_meta['geometry'] = geo_point

em_point_gdf = gpd.GeoDataFrame(em_meta, geometry= 'geometry')
em_point_gdf.crs = {'init':'epsg:4326'}

url = 'http://apis.data.go.kr/B552584/MsrstnInfoInqireSvc/getMsrstnList'
key = "k5wXUhoJHwee1cncQCBmm81YbQ+exttb0vdJcyF5GuGJn0mbGBNNL/ER2VfkrJMlExfc+FZjPeRuOM2bvgDYyQ=="

# params ={'serviceKey' : key, 'addr': '서울', 'stationName': '종로구', 'pageNo': 1, 'numOfRows': 10,'returnType': 'json'}
params ={'serviceKey' : key, 'pageNo': 1, 'numOfRows': 640,'returnType': 'json'}
response = requests.get(url, params=params, verify=False)

data_dic = response.json()
site_info = pd.DataFrame(data_dic['response']['body']['items'])

point_list = []
for i in range(len(site_info)):
    try:
        point_list.append(Point(float(site_info.dmY[i]), float(site_info.dmX[i])))
    except:
        point_list.append(None)

site_info.loc[:,'geometry'] = point_list
site_info_dropna = site_info.loc[~site_info.isna().dmX.values]

site_info_dropna = gpd.GeoDataFrame(site_info_dropna, geometry = 'geometry')
site_info_dropna.crs = em_point_gdf.crs

/usr/local/lib/python3.8/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [82]:
import datetime 
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.merge import merge
import tensorflow as tf
import xarray as xr

In [83]:
# 대기질자료 불러오기
airpolutant_monitoring_2013_list = glob(os.path.join(data_path,"시간단위대기질자료","rawdata","extract","2013*",))
airpolutant_monitoring_2013_list
data_list = []
for i in airpolutant_monitoring_2013_list:
    temp_data = pd.read_excel(i)
    data_list.append(temp_data)
air_pol_2013 = pd.concat(data_list)
air_pol_2013.info()

#기상자료 불러오기
em_dataset_path_list = glob(os.path.join(data_path,"시간단위_기상자료개방포털","rawdata","extract","*.csv",))
em_dataset_list = [pd.read_csv(path, encoding='cp949') for path in em_dataset_path_list]
em_df = pd.concat(em_dataset_list, axis = 0)
em_df.info()
datetime_airpol = [datetime.datetime(int(date[:4]),int(date[4:6]),int(date[6:8]),int(date[8:] if date[8:] !='24' else '0')) for date in air_pol_2013.측정일시.values.astype(str)]
air_pol_2013.loc[:,'datetime'] = datetime_airpol
em_df.loc[:,'datetime'] = pd.to_datetime(em_df.일시)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2766000 entries, 0 to 699935
Data columns (total 10 columns):
 #   Column  Dtype  
---  ------  -----  
 0   지역      object 
 1   측정소명    object 
 2   측정소코드   int64  
 3   측정일시    int64  
 4   SO2     float64
 5   CO      float64
 6   O3      float64
 7   NO2     float64
 8   PM10    int64  
 9   주소      object 
dtypes: float64(4), int64(3), object(3)
memory usage: 232.1+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 262839 entries, 0 to 8727
Data columns (total 27 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   지점             262839 non-null  int64  
 1   일시             262839 non-null  object 
 2   기온(°C)         262609 non-null  float64
 3   강수량(mm)        19578 non-null   float64
 4   풍속(m/s)        262522 non-null  float64
 5   풍향(16방위)       262520 non-null  float64
 6   습도(%)          262609 non-null  float64
 7   증기압(hPa)       262610 non-null  float64
 8   이슬점온도(°C

In [84]:
# out_grid = make_geocube(vector_data=test_data, measurements=["습도(%)"], geom=geom,resolution=(9000, 9000), fill=0, output_crs=projout) #for most crs negative comes first in resolution
x_m = list(range(-180000,-180000 + 9000 * 68, 9000))
y_m = list(range(-585000,-585000 + 9000 * 83, 9000))

print(len(x_m), len(y_m))

grid_points = []
for x_i in x_m:
    for y_i in y_m:
        grid_points.append(Point(x_i,y_i))

grid_data = pd.DataFrame(grid_points, columns=['geometry'])

grid_data = gpd.GeoDataFrame(grid_data, geometry='geometry')
grid_data.crs = em_point_gdf.to_crs(projout).crs
grid_data.loc[:,'x_m'] = grid_data.geometry.x
grid_data.loc[:,'y_m'] = grid_data.geometry.y
grid_data.loc[:,'value'] = 0

out_grid = make_geocube(vector_data=grid_data, measurements=["value"],resolution=(9000, 9000), fill=0, output_crs=projout) #for most crs negative comes first in resolution
out_grid["value"].rio.to_raster(os.path.join(record_path,"base_lcc.tif"))
# 레스터 좌표변환
# 5181로 변환하면 좌표들 약간 틀어짐
# lcc로 처리 해야함
from rasterio.warp import calculate_default_transform, reproject, Resampling
# dst_crs = 'epsg: 5181'
dst_crs = '+proj=lcc +lat_1=30 +lat_2=60 +lon_1=126 +lat_0=38 +lon_0=126 +ellps=GRS80 +units=m'

with rasterio.open(os.path.join(record_path,"base_lcc.tif")) as src:

    transform, width, height = calculate_default_transform(
                    src.crs, 
                    dst_crs, 
                    src.width, 
                    src.height, 
                    *src.bounds)
   
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    with rasterio.open(os.path.join(record_path,"base_lcc_lcc.tif"), 'w', **kwargs) as dst:
            
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)

68 83


In [85]:
##################밴드 뎁스 레스터로 만들기( 머지할때 필요 )

# out_grid = make_geocube(vector_data=test_data, measurements=["습도(%)"], geom=geom,resolution=(9000, 9000), fill=0, output_crs=projout) #for most crs negative comes first in resolution
x_m = list(range(-180000,-180000 + 9000 * 68, 9000))
y_m = list(range(-585000,-585000 + 9000 * 83, 9000))

print(len(x_m), len(y_m))

grid_points = []
for x_i in x_m:
    for y_i in y_m:
        grid_points.append(Point(x_i,y_i))

grid_data = pd.DataFrame(grid_points, columns=['geometry'])

grid_data = gpd.GeoDataFrame(grid_data, geometry='geometry')
grid_data.crs = em_point_gdf.to_crs(projout).crs
grid_data.loc[:,'x_m'] = grid_data.geometry.x
grid_data.loc[:,'y_m'] = grid_data.geometry.y
grid_data.loc[:,'value'] = 0

out_grid = make_geocube(vector_data=grid_data, measurements=["value"],resolution=(9000, 9000), fill=0, output_crs=projout) #for most crs negative comes first in resolution
out_grid["value"].rio.to_raster(os.path.join(record_path,"base_lcc.tif"))
# 레스터 좌표변환
# 5181로 변환하면 좌표들 약간 틀어짐
# lcc로 처리 해야함

# dst_crs = 'epsg: 5181'
dst_crs = '+proj=lcc +lat_1=30 +lat_2=60 +lon_1=126 +lat_0=38 +lon_0=126 +ellps=GRS80 +units=m'

with rasterio.open(os.path.join(record_path,"base_lcc.tif")) as src:

    transform, width, height = calculate_default_transform(
                    src.crs, 
                    dst_crs, 
                    src.width, 
                    src.height, 
                    *src.bounds)
   
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height,
        'count': band_depth,
        
    })

    with rasterio.open(os.path.join(record_path,f"base_lcc_lcc_{band_depth}depth.tif"), 'w', **kwargs) as dst:
            
            for i in range(1, band_depth + 1):
                reproject(
                    source=rasterio.band(src, 1),
                    destination=rasterio.band(dst, 1),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)

68 83


* version1

In [ ]:
for i in senario_list:

    if i == 'RSM_1':
        print(i, '건너뛰기')
        continue

    print(i)

    path_emis = emission_path_list[np.array([path.split("/")[-1] for path in emission_path_list]) == i][0]
    path_conc = concentration_path_list[np.array([path.split("/")[-1] for path in concentration_path_list]) == i][0]
    s_num = i.split("_")[-1]
    
    # 날짜별 smoke, cmaq데이터셋 경로 정리
    emis_list = glob(os.path.join(path_emis,"*"))         
    concentration_list = glob(os.path.join(path_conc,"*"))           

    emis_day_list = [os.path.split(path)[-1].split('.')[-5] for path in emis_list]
    concentration_day_list = [os.path.split(path)[-1].split('.')[-2] for path in concentration_list]

    
    concentration_day_list_datetime = []
    for date in concentration_day_list:

        start_date = datetime.date(int(date[:4]), 1, 1) + datetime.timedelta(-1)
        d_day = int(date[4:])
        target_date = start_date + datetime.timedelta(d_day)

        concentration_day_list_datetime.append(target_date)

    emis_day_list_datetime = []
    for date in emis_day_list:
        target_date = datetime.date(int(date[:4]), int(date[4:6]), int(date[6:]))
        emis_day_list_datetime.append(target_date)

    emis_info_df = pd.DataFrame()
    emis_info_df.loc[:,['date']] = emis_day_list_datetime
    emis_info_df.loc[:,['path']] = emis_list

    concentration_info_df = pd.DataFrame()
    concentration_info_df.loc[:,['date']] = concentration_day_list_datetime
    concentration_info_df.loc[:,['path']] = concentration_list
    concentration_info_df = concentration_info_df.sort_values('date')
    concentration_info_df.index = range(len(concentration_info_df))

    con_emis_info_merged_df = pd.merge(concentration_info_df, emis_info_df, how='left', on='date', suffixes=['_concentration', '_emission'])
        
    con_emis_info_merged_df_dropna = con_emis_info_merged_df.loc[~con_emis_info_merged_df.path_emission.isna(),]
    con_emis_info_merged_df_dropna.index = range(len(con_emis_info_merged_df_dropna))

    writer_image = tf.io.TFRecordWriter(os.path.join(record_path,f"dataset_{s_num}.tfr"))
    print(os.path.join(record_path,f"dataset_{s_num}.tfr"))

    for k in range(len(con_emis_info_merged_df_dropna)):
    # for i in range(len(con_emis_info_merged_df_dropna))[:1]:
    
        today = con_emis_info_merged_df_dropna.loc[k].date
        print(today)
        # 해당날짜의 데이터셋 불러오기
        with Dataset(con_emis_info_merged_df_dropna.loc[k].path_concentration, 'r') as concen_data, Dataset(con_emis_info_merged_df_dropna.loc[k].path_emission, 'r') as emis_data:

            # concen_data = Dataset(con_emis_info_merged_df_dropna.loc[k].path_concentration, 'r')
            # emis_data = Dataset(con_emis_info_merged_df_dropna.loc[k].path_emission, 'r')

            # 해당날짜의 관련변수 합으로 pm10 CMAQ array형성
            pm10_chemlist = [np.array(concen_data.variables[chem]) for chem in pm_10_chem_list]
            day_concen_pm10 = np.sum(pm10_chemlist, axis=0)
            day_concen_pm10 = day_concen_pm10.reshape(day_concen_pm10.shape[0],day_concen_pm10.shape[2],day_concen_pm10.shape[3],-1)

            # 해당날짜의 smoke 모든변수 array 형성, 연직방향으로는 맨 밑레이어만 사용
            smoke_chem_list = list(set(emis_data.variables.keys()) - set(['TFLAG']))
            # smoke_chem_arr_list = [np.array(emis_data.variables[chem])[:,0:1,:,:].reshape(25,82,67,-1) for chem in smoke_chem_list]
            smoke_chem_arr_list = [emis_data[chem][:,0,:,:].reshape(25,82,67,-1) for chem in smoke_chem_list]
            day_smoke_allval = np.concatenate(smoke_chem_arr_list, axis=3)
            day_smoke_allval = day_smoke_allval[:24]

            print('CMAQ_PM10 데이터셋: ',day_concen_pm10.shape, 'SMOKE_모든변수 데이터셋: ',day_smoke_allval.shape)

            for hour_ind in range(24):
            # for hour_ind in range(4):
                datetime_ind = datetime.datetime(today.year, today.month, today.day, hour_ind)
                print(str(datetime_ind))

                air_pol_datetime_ind_df = air_pol_2013.loc[air_pol_2013.datetime == datetime_ind]
                air_pol_datetime_ind_df.index = range(len(air_pol_datetime_ind_df))

                air_pol_datetime_ind_df_geoinfo = pd.merge(air_pol_datetime_ind_df,site_info_dropna.loc[:,['stationName', 'geometry']],how='left',left_on='측정소명',right_on='stationName')
                air_pol_datetime_ind_df_geoinfo = air_pol_datetime_ind_df_geoinfo.loc[~air_pol_datetime_ind_df_geoinfo.stationName.isna()]
                air_pol_datetime_ind_df_geoinfo.index = range(len(air_pol_datetime_ind_df_geoinfo))
                air_pol_datetime_ind_df_geoinfo = gpd.GeoDataFrame(air_pol_datetime_ind_df_geoinfo, geometry='geometry')
                air_pol_datetime_ind_df_geoinfo = air_pol_datetime_ind_df_geoinfo.fillna(0)
                air_pol_datetime_ind_df_geoinfo = air_pol_datetime_ind_df_geoinfo.loc[~(air_pol_datetime_ind_df_geoinfo.PM10 == -999)]  # 값이 -999찍혀있을때가 있음 기기오작동으로보고 제거
                air_pol_datetime_ind_df_geoinfo.index = range(len(air_pol_datetime_ind_df_geoinfo))

                # out_grid = make_geocube(vector_data=test_data, measurements=["습도(%)"], geom=geom,resolution=(9000, 9000), fill=0, output_crs=projout) #for most crs negative comes first in resolution
                out_grid = make_geocube(vector_data=air_pol_datetime_ind_df_geoinfo, measurements=["PM10"],resolution=(9000, 9000), fill=0, output_crs=projout) #for most crs negative comes first in resolution
                out_grid["PM10"].rio.to_raster(os.path.join(record_path,"rasterized_air_pol.tif"))


                dst_crs = projout
                with rasterio.open(os.path.join(record_path,"rasterized_air_pol.tif")) as src:
                    # print(src.width, src.height, *src.bounds,)
                    transform, width, height = calculate_default_transform(
                                    src.crs, 
                                    dst_crs, 
                                    src.width, 
                                    src.height, 
                                    *src.bounds)
                
                    kwargs = src.meta.copy()
                    kwargs.update({
                        'crs': dst_crs,
                        'transform': transform,
                        'width': width,
                        'height': height
                    })

                    with rasterio.open(os.path.join(record_path,"rasterized_air_pol_lcc.tif"), 'w', **kwargs) as dst:
                            
                            for i in range(1, src.count + 1):
                                reproject(
                                    source=rasterio.band(src, i),
                                    destination=rasterio.band(dst, i),
                                    src_transform=src.transform,
                                    src_crs=src.crs,
                                    dst_transform=transform,
                                    dst_crs=dst_crs,
                                    resampling=Resampling.nearest)

            

                with rasterio.open(os.path.join(record_path,"base_lcc_lcc.tif")) as src, rasterio.open(os.path.join(record_path,"rasterized_air_pol_lcc.tif")) as src2:
                    raster_list = [src, src2]
                    # print(src.read(1).shape, src2.read(1).shape,)
                    mosaic, output = merge(raster_list)

                    output_meta = src2.meta.copy()
                    output_meta.update(
                        {"driver": "GTiff",
                            "height": mosaic.shape[1],
                            "width": mosaic.shape[2],
                            "transform": output,
                        }
                    )

                    with rasterio.open(os.path.join(record_path,"rasterized_air_pol_lcc_final.tif"), "w", **output_meta) as m:
                        m.write(mosaic)

                with rasterio.open(os.path.join(record_path,"rasterized_air_pol_lcc_final.tif")) as src:
                    air_pol_pm10_arr = src.read(1)
                    air_pol_pm10_arr = air_pol_pm10_arr.reshape(1,82,67,1)
                    # print(air_pol_pm10_arr.shape)
                em_df_ind_df = em_df.loc[em_df.datetime == datetime_ind]
                em_df_ind_df.index = range(len(em_df_ind_df))
                em_df_ind_df_geoinfo = pd.merge(em_df_ind_df,em_point_gdf.loc[:,['지점', 'geometry']],how='left',on = '지점')

                em_df_ind_df_geoinfo = em_df_ind_df_geoinfo.loc[~em_df_ind_df_geoinfo.geometry.isna()]
                em_df_ind_df_geoinfo.index = range(len(em_df_ind_df_geoinfo))
                em_df_ind_df_geoinfo = gpd.GeoDataFrame(em_df_ind_df_geoinfo, geometry='geometry')
                em_df_ind_df_geoinfo = em_df_ind_df_geoinfo.fillna(0)


                out_grid = make_geocube(vector_data=em_df_ind_df_geoinfo, measurements=col_list,resolution=(9000, 9000), fill=0, output_crs=projout) #for most crs negative comes first in resolution
                # out_grid[col_name].rio.to_raster(os.path.join(record_path,"rasterized_em.tif"))
                out_grid.rio.to_raster(os.path.join(record_path,"rasterized_em.tif"))
                with rasterio.open(os.path.join(record_path,"rasterized_em.tif")) as src:
                        # print(src.count)
                        transform, width, height = calculate_default_transform(
                                        src.crs, 
                                        dst_crs, 
                                        src.width, 
                                        src.height, 
                                        *src.bounds)
                    
                        kwargs = src.meta.copy()
                        kwargs.update({
                            'crs': dst_crs,
                            'transform': transform,
                            'width': width,
                            'height': height
                        })

                    

                        with rasterio.open(os.path.join(record_path,"rasterized_em_lcc.tif"), 'w', **kwargs) as dst:

                            for i in range(1, src.count + 1):
                                
                                reproject(
                                    source=rasterio.band(src, i),
                                    destination=rasterio.band(dst, i),
                                    src_transform=src.transform,
                                    src_crs=src.crs,
                                    dst_transform=transform,
                                    dst_crs=dst_crs,
                                    resampling=Resampling.nearest)

                with rasterio.open(os.path.join(record_path,f"base_lcc_lcc_{band_depth}depth.tif")) as src, rasterio.open(os.path.join(record_path,"rasterized_em_lcc.tif")) as src2:
                        raster_list = [src, src2]
                        # print(src.count, src2.count,)
                        mosaic, output = merge(raster_list,)
                        
                        output_meta = src2.meta.copy()
                        output_meta.update(
                            {"driver": "GTiff",
                                "height": mosaic.shape[1],
                                "width": mosaic.shape[2],
                                "transform": output,
                            }
                        )

                        with rasterio.open(os.path.join(record_path,"rasterized_em_lcc_final.tif"), "w", **output_meta) as m:
                            m.write(mosaic)

                
                with rasterio.open(os.path.join(record_path,"rasterized_em_lcc_final.tif")) as src:

                    em_arr_allband = src.read()
                    em_arr_map = em_arr_allband.reshape(1,82,67,-1)

                weather_t = em_arr_map[:1].tobytes()
                air_q_t = air_pol_pm10_arr[:1].tobytes()
                cmaq_t = day_concen_pm10[hour_ind:hour_ind+1].tobytes()
                smoke_t = day_smoke_allval[hour_ind:hour_ind+1].tobytes()

                example = tf.train.Example(
                    features=tf.train.Features(
                        feature={
                            'CMAQ_t': _bytes_feature(cmaq_t),
                            'SMOKE_t': _bytes_feature(smoke_t),
                            'air_quality_monitoring_t': _bytes_feature(air_q_t),   
                            'weather_monitoring_t': _bytes_feature(weather_t),
                            'year': _bytes_feature(np.array(today.year - 2000).reshape(1,-1).tobytes()), 
                            'month': _bytes_feature(np.array(today.month).reshape(1,-1).tobytes()), 
                            'day': _bytes_feature(np.array(today.day).reshape(1,-1).tobytes()), 
                            'hour': _bytes_feature(np.array(hour_ind).reshape(1,-1).tobytes()), 

                        }
                        )
                    )
                writer_image.write(example.SerializeToString())
        
    writer_image.close()



    

* version2
* xr 사용

In [234]:
## version2

for i in senario_list:

    if i == 'RSM_1':
        print(i, '건너뛰기')
        continue

    print(i)

    path_emis = emission_path_list[np.array([path.split("/")[-1] for path in emission_path_list]) == i][0]
    path_conc = concentration_path_list[np.array([path.split("/")[-1] for path in concentration_path_list]) == i][0]
    s_num = i.split("_")[-1]
    
    # 날짜별 smoke, cmaq데이터셋 경로 정리
    emis_list = glob(os.path.join(path_emis,"*"))         
    concentration_list = glob(os.path.join(path_conc,"*"))           

    emis_day_list = [os.path.split(path)[-1].split('.')[-5] for path in emis_list]
    concentration_day_list = [os.path.split(path)[-1].split('.')[-2] for path in concentration_list]

    
    concentration_day_list_datetime = []
    for date in concentration_day_list:

        start_date = datetime.date(int(date[:4]), 1, 1) + datetime.timedelta(-1)
        d_day = int(date[4:])
        target_date = start_date + datetime.timedelta(d_day)

        concentration_day_list_datetime.append(target_date)

    emis_day_list_datetime = []
    for date in emis_day_list:
        target_date = datetime.date(int(date[:4]), int(date[4:6]), int(date[6:]))
        emis_day_list_datetime.append(target_date)

    emis_info_df = pd.DataFrame()
    emis_info_df.loc[:,['date']] = emis_day_list_datetime
    emis_info_df.loc[:,['path']] = emis_list

    concentration_info_df = pd.DataFrame()
    concentration_info_df.loc[:,['date']] = concentration_day_list_datetime
    concentration_info_df.loc[:,['path']] = concentration_list
    concentration_info_df = concentration_info_df.sort_values('date')
    concentration_info_df.index = range(len(concentration_info_df))

    con_emis_info_merged_df = pd.merge(concentration_info_df, emis_info_df, how='left', on='date', suffixes=['_concentration', '_emission'])
        
    con_emis_info_merged_df_dropna = con_emis_info_merged_df.loc[~con_emis_info_merged_df.path_emission.isna(),]
    con_emis_info_merged_df_dropna.index = range(len(con_emis_info_merged_df_dropna))

    writer_image = tf.io.TFRecordWriter(os.path.join(record_path,f"dataset_{s_num}.tfr"))
    print(os.path.join(record_path,f"dataset_{s_num}.tfr"))

    for k in range(len(con_emis_info_merged_df_dropna)):
    # for i in range(len(con_emis_info_merged_df_dropna))[:1]:
    
        today = con_emis_info_merged_df_dropna.loc[k].date
        print(today)
        # 해당날짜의 데이터셋 불러오기
        with xr.open_dataset(con_emis_info_merged_df_dropna.loc[k].path_concentration) as concen_data, xr.open_dataset(con_emis_info_merged_df_dropna.loc[k].path_emission) as emis_data:

            pm10_chem_map = list(map(lambda chem: concen_data[chem], pm_10_chem_list))
            smoke_chem_list = list(set(emis_data.variables.keys()) - set(['TFLAG']))
            
            smoke_chem_map = list(map(lambda chem: emis_data[chem][:,0,:,:], smoke_chem_list))

            day_smoke_allval = np.moveaxis(np.array(smoke_chem_map), 0,3)
            day_smoke_allval = day_smoke_allval[:24]

            print('CMAQ_PM10 데이터셋: ',day_concen_pm10.shape, 'SMOKE_모든변수 데이터셋: ',day_smoke_allval.shape)

            for hour_ind in range(24):
            # for hour_ind in range(4):
                datetime_ind = datetime.datetime(today.year, today.month, today.day, hour_ind)
                print(str(datetime_ind))

                air_pol_datetime_ind_df = air_pol_2013.loc[air_pol_2013.datetime == datetime_ind]
                air_pol_datetime_ind_df.index = range(len(air_pol_datetime_ind_df))

                air_pol_datetime_ind_df_geoinfo = pd.merge(air_pol_datetime_ind_df,site_info_dropna.loc[:,['stationName', 'geometry']],how='left',left_on='측정소명',right_on='stationName')
                air_pol_datetime_ind_df_geoinfo = air_pol_datetime_ind_df_geoinfo.loc[~air_pol_datetime_ind_df_geoinfo.stationName.isna()]
                air_pol_datetime_ind_df_geoinfo.index = range(len(air_pol_datetime_ind_df_geoinfo))
                air_pol_datetime_ind_df_geoinfo = gpd.GeoDataFrame(air_pol_datetime_ind_df_geoinfo, geometry='geometry')
                air_pol_datetime_ind_df_geoinfo = air_pol_datetime_ind_df_geoinfo.fillna(0)
                air_pol_datetime_ind_df_geoinfo = air_pol_datetime_ind_df_geoinfo.loc[~(air_pol_datetime_ind_df_geoinfo.PM10 == -999)]  # 값이 -999찍혀있을때가 있음 기기오작동으로보고 제거
                air_pol_datetime_ind_df_geoinfo.index = range(len(air_pol_datetime_ind_df_geoinfo))

                # out_grid = make_geocube(vector_data=test_data, measurements=["습도(%)"], geom=geom,resolution=(9000, 9000), fill=0, output_crs=projout) #for most crs negative comes first in resolution
                out_grid = make_geocube(vector_data=air_pol_datetime_ind_df_geoinfo, measurements=["PM10"],resolution=(9000, 9000), fill=0, output_crs=projout) #for most crs negative comes first in resolution
                out_grid["PM10"].rio.to_raster(os.path.join(record_path,"rasterized_air_pol.tif"))


                dst_crs = projout
                with rasterio.open(os.path.join(record_path,"rasterized_air_pol.tif")) as src:
                    # print(src.width, src.height, *src.bounds,)
                    transform, width, height = calculate_default_transform(
                                    src.crs, 
                                    dst_crs, 
                                    src.width, 
                                    src.height, 
                                    *src.bounds)
                
                    kwargs = src.meta.copy()
                    kwargs.update({
                        'crs': dst_crs,
                        'transform': transform,
                        'width': width,
                        'height': height
                    })

                    with rasterio.open(os.path.join(record_path,"rasterized_air_pol_lcc.tif"), 'w', **kwargs) as dst:
                            
                            for i in range(1, src.count + 1):
                                reproject(
                                    source=rasterio.band(src, i),
                                    destination=rasterio.band(dst, i),
                                    src_transform=src.transform,
                                    src_crs=src.crs,
                                    dst_transform=transform,
                                    dst_crs=dst_crs,
                                    resampling=Resampling.nearest)

            

                with rasterio.open(os.path.join(record_path,"base_lcc_lcc.tif")) as src, rasterio.open(os.path.join(record_path,"rasterized_air_pol_lcc.tif")) as src2:
                    raster_list = [src, src2]
                    # print(src.read(1).shape, src2.read(1).shape,)
                    mosaic, output = merge(raster_list)

                    output_meta = src2.meta.copy()
                    output_meta.update(
                        {"driver": "GTiff",
                            "height": mosaic.shape[1],
                            "width": mosaic.shape[2],
                            "transform": output,
                        }
                    )

                    with rasterio.open(os.path.join(record_path,"rasterized_air_pol_lcc_final.tif"), "w", **output_meta) as m:
                        m.write(mosaic)

                with rasterio.open(os.path.join(record_path,"rasterized_air_pol_lcc_final.tif")) as src:
                    air_pol_pm10_arr = src.read(1)
                    air_pol_pm10_arr = air_pol_pm10_arr.reshape(1,82,67,1)
                    # print(air_pol_pm10_arr.shape)
                em_df_ind_df = em_df.loc[em_df.datetime == datetime_ind]
                em_df_ind_df.index = range(len(em_df_ind_df))
                em_df_ind_df_geoinfo = pd.merge(em_df_ind_df,em_point_gdf.loc[:,['지점', 'geometry']],how='left',on = '지점')

                em_df_ind_df_geoinfo = em_df_ind_df_geoinfo.loc[~em_df_ind_df_geoinfo.geometry.isna()]
                em_df_ind_df_geoinfo.index = range(len(em_df_ind_df_geoinfo))
                em_df_ind_df_geoinfo = gpd.GeoDataFrame(em_df_ind_df_geoinfo, geometry='geometry')
                em_df_ind_df_geoinfo = em_df_ind_df_geoinfo.fillna(0)


                out_grid = make_geocube(vector_data=em_df_ind_df_geoinfo, measurements=col_list,resolution=(9000, 9000), fill=0, output_crs=projout) #for most crs negative comes first in resolution
                # out_grid[col_name].rio.to_raster(os.path.join(record_path,"rasterized_em.tif"))
                out_grid.rio.to_raster(os.path.join(record_path,"rasterized_em.tif"))
                with rasterio.open(os.path.join(record_path,"rasterized_em.tif")) as src:
                        # print(src.count)
                        transform, width, height = calculate_default_transform(
                                        src.crs, 
                                        dst_crs, 
                                        src.width, 
                                        src.height, 
                                        *src.bounds)
                    
                        kwargs = src.meta.copy()
                        kwargs.update({
                            'crs': dst_crs,
                            'transform': transform,
                            'width': width,
                            'height': height
                        })

                    

                        with rasterio.open(os.path.join(record_path,"rasterized_em_lcc.tif"), 'w', **kwargs) as dst:

                            for i in range(1, src.count + 1):
                                
                                reproject(
                                    source=rasterio.band(src, i),
                                    destination=rasterio.band(dst, i),
                                    src_transform=src.transform,
                                    src_crs=src.crs,
                                    dst_transform=transform,
                                    dst_crs=dst_crs,
                                    resampling=Resampling.nearest)

                with rasterio.open(os.path.join(record_path,f"base_lcc_lcc_{band_depth}depth.tif")) as src, rasterio.open(os.path.join(record_path,"rasterized_em_lcc.tif")) as src2:
                        raster_list = [src, src2]
                        # print(src.count, src2.count,)
                        mosaic, output = merge(raster_list,)
                        
                        output_meta = src2.meta.copy()
                        output_meta.update(
                            {"driver": "GTiff",
                                "height": mosaic.shape[1],
                                "width": mosaic.shape[2],
                                "transform": output,
                            }
                        )

                        with rasterio.open(os.path.join(record_path,"rasterized_em_lcc_final.tif"), "w", **output_meta) as m:
                            m.write(mosaic)

                
                with rasterio.open(os.path.join(record_path,"rasterized_em_lcc_final.tif")) as src:

                    em_arr_allband = src.read()
                    em_arr_map = em_arr_allband.reshape(1,82,67,-1)

                weather_t = em_arr_map[:1].tobytes()
                air_q_t = air_pol_pm10_arr[:1].tobytes()
                cmaq_t = day_concen_pm10[hour_ind:hour_ind+1].tobytes()
                smoke_t = day_smoke_allval[hour_ind:hour_ind+1].tobytes()

                example = tf.train.Example(
                    features=tf.train.Features(
                        feature={
                            'CMAQ_t': _bytes_feature(cmaq_t),
                            'SMOKE_t': _bytes_feature(smoke_t),
                            'air_quality_monitoring_t': _bytes_feature(air_q_t),   
                            'weather_monitoring_t': _bytes_feature(weather_t),
                            'year': _bytes_feature(np.array(today.year - 2000).reshape(1,-1).tobytes()), 
                            'month': _bytes_feature(np.array(today.month).reshape(1,-1).tobytes()), 
                            'day': _bytes_feature(np.array(today.day).reshape(1,-1).tobytes()), 
                            'hour': _bytes_feature(np.array(hour_ind).reshape(1,-1).tobytes()), 

                        }
                        )
                    )
                writer_image.write(example.SerializeToString())
        
    writer_image.close()



    

RSM_89
/home/workspace/cmaqProjectdata/dataset_record/dataset_89.tfr
2013-03-22
CMAQ_PM10 데이터셋:  (24, 82, 67, 1) SMOKE_모든변수 데이터셋:  (24, 82, 67, 45)
2013-03-22 00:00:00
2013-03-22 01:00:00
2013-03-22 02:00:00
2013-03-22 03:00:00
2013-03-22 04:00:00
2013-03-22 05:00:00
2013-03-22 06:00:00
2013-03-22 07:00:00
2013-03-22 08:00:00
2013-03-22 09:00:00
2013-03-22 10:00:00
2013-03-22 11:00:00
2013-03-22 12:00:00
2013-03-22 13:00:00
2013-03-22 14:00:00
2013-03-22 15:00:00
2013-03-22 16:00:00
2013-03-22 17:00:00
2013-03-22 18:00:00
2013-03-22 19:00:00
2013-03-22 20:00:00
2013-03-22 21:00:00
2013-03-22 22:00:00
2013-03-22 23:00:00
2013-03-23
CMAQ_PM10 데이터셋:  (24, 82, 67, 1) SMOKE_모든변수 데이터셋:  (24, 82, 67, 45)
2013-03-23 00:00:00
2013-03-23 01:00:00
2013-03-23 02:00:00
2013-03-23 03:00:00
2013-03-23 04:00:00
2013-03-23 05:00:00
2013-03-23 06:00:00
2013-03-23 07:00:00
2013-03-23 08:00:00
2013-03-23 09:00:00
2013-03-23 10:00:00
2013-03-23 11:00:00
2013-03-23 12:00:00
2013-03-23 13:00:00
2013-03-23 14

In [ ]:
%%time
for i in range(1000):
    start_t = time.time()
    print(i, time.time())
    with Dataset(con_emis_info_merged_df_dropna.loc[k].path_concentration, 'r') as concen_data, Dataset(con_emis_info_merged_df_dropna.loc[k].path_emission, 'r') as emis_data:

            # concen_data = Dataset(con_emis_info_merged_df_dropna.loc[k].path_concentration, 'r')
            # emis_data = Dataset(con_emis_info_merged_df_dropna.loc[k].path_emission, 'r')

            # 해당날짜의 관련변수 합으로 pm10 CMAQ array형성
            pm10_chemlist = [np.array(concen_data.variables[chem]) for chem in pm_10_chem_list]
            day_concen_pm10 = np.sum(pm10_chemlist, axis=0)
            day_concen_pm10 = day_concen_pm10.reshape(day_concen_pm10.shape[0],day_concen_pm10.shape[2],day_concen_pm10.shape[3],-1)

            # 해당날짜의 smoke 모든변수 array 형성, 연직방향으로는 맨 밑레이어만 사용
            smoke_chem_list = list(set(emis_data.variables.keys()) - set(['TFLAG']))
            # smoke_chem_arr_list = [np.array(emis_data.variables[chem])[:,0:1,:,:].reshape(25,82,67,-1) for chem in smoke_chem_list]
            smoke_chem_arr_list = [emis_data[chem][:,0,:,:].reshape(25,82,67,-1) for chem in smoke_chem_list]
    end_t = time.time()
    print(end_t - start_t)

In [ ]:
conc_data = xr.open_dataset(merged_df.loc[i].path_concentration)
emis_data = xr.open_dataset(merged_df.loc[i].path_emission)

pm10_chem_map = map(lambda chem: conc_data[chem], pm_10_chem_list)
smoke_chem_list = list(set(emis_data.variables.keys()) - set(['TFLAG']))
print(time.time())
smoke_chem_map = map(lambda chem: emis_data[chem], smoke_chem_list)
print(time.time())

In [194]:
smoke_chem_map

In [192]:

import xarray as xr

In [ ]:
%%time
for i in range(1000):
    start_t = time.time()
    print(i, time.time())
    with xr.open_dataset(con_emis_info_merged_df_dropna.loc[k].path_concentration) as concen_data, xr.open_dataset(con_emis_info_merged_df_dropna.loc[k].path_emission) as emis_data:

            # concen_data = Dataset(con_emis_info_merged_df_dropna.loc[k].path_concentration, 'r')
            # emis_data = Dataset(con_emis_info_merged_df_dropna.loc[k].path_emission, 'r')

            # # 해당날짜의 관련변수 합으로 pm10 CMAQ array형성
            # pm10_chemlist = [np.array(concen_data.variables[chem]) for chem in pm_10_chem_list]
            # day_concen_pm10 = np.sum(pm10_chemlist, axis=0)
            # day_concen_pm10 = day_concen_pm10.reshape(day_concen_pm10.shape[0],day_concen_pm10.shape[2],day_concen_pm10.shape[3],-1)

            # # 해당날짜의 smoke 모든변수 array 형성, 연직방향으로는 맨 밑레이어만 사용
            # smoke_chem_list = list(set(emis_data.variables.keys()) - set(['TFLAG']))
            # # smoke_chem_arr_list = [np.array(emis_data.variables[chem])[:,0:1,:,:].reshape(25,82,67,-1) for chem in smoke_chem_list]
            # smoke_chem_arr_list = [emis_data[chem][:,0,:,:].reshape(25,82,67,-1) for chem in smoke_chem_list]

            pm10_chem_map = list(map(lambda chem: concen_data[chem], pm_10_chem_list))
            smoke_chem_list = list(set(emis_data.variables.keys()) - set(['TFLAG']))
            
            smoke_chem_map = list(map(lambda chem: emis_data[chem][:,0,:,:], smoke_chem_list))

            np.array(smoke_chem_map).shape

        np.moveaxis(np.array(smoke_chem_map), 0,3)
            
    end_t = time.time()
    print(end_t - start_t)

In [228]:
np.array(smoke_chem_map).shape

np.moveaxis(np.array(smoke_chem_map), 0,3)

(45, 25, 82, 67)

In [233]:
np.min(np.moveaxis(np.array(smoke_chem_map), 0,3)[0,:,:,0] - np.array(smoke_chem_map)[0,0,:,:])

0.0

In [221]:
smoke_chem_map[0]

<xarray.DataArray 'PNO3' (TSTEP: 25, ROW: 82, COL: 67)>
array([[[0., 0., ..., 0., 0.],
        [0., 0., ..., 0., 0.],
        ...,
        [0., 0., ..., 0., 0.],
        [0., 0., ..., 0., 0.]],

       [[0., 0., ..., 0., 0.],
        [0., 0., ..., 0., 0.],
        ...,
        [0., 0., ..., 0., 0.],
        [0., 0., ..., 0., 0.]],

       ...,

       [[0., 0., ..., 0., 0.],
        [0., 0., ..., 0., 0.],
        ...,
        [0., 0., ..., 0., 0.],
        [0., 0., ..., 0., 0.]],

       [[0., 0., ..., 0., 0.],
        [0., 0., ..., 0., 0.],
        ...,
        [0., 0., ..., 0., 0.],
        [0., 0., ..., 0., 0.]]], dtype=float32)
Dimensions without coordinates: TSTEP, ROW, COL
Attributes:
    long_name:  PNO3            
    units:      g/s             
    var_desc:   Model species PNO3                                           ...

In [214]:
emis_data[chem][:,0,:,:]

<xarray.DataArray 'PNO3' (TSTEP: 25, ROW: 82, COL: 67)>
array([[[0., 0., ..., 0., 0.],
        [0., 0., ..., 0., 0.],
        ...,
        [0., 0., ..., 0., 0.],
        [0., 0., ..., 0., 0.]],

       [[0., 0., ..., 0., 0.],
        [0., 0., ..., 0., 0.],
        ...,
        [0., 0., ..., 0., 0.],
        [0., 0., ..., 0., 0.]],

       ...,

       [[0., 0., ..., 0., 0.],
        [0., 0., ..., 0., 0.],
        ...,
        [0., 0., ..., 0., 0.],
        [0., 0., ..., 0., 0.]],

       [[0., 0., ..., 0., 0.],
        [0., 0., ..., 0., 0.],
        ...,
        [0., 0., ..., 0., 0.],
        [0., 0., ..., 0., 0.]]], dtype=float32)
Dimensions without coordinates: TSTEP, ROW, COL
Attributes:
    long_name:  PNO3            
    units:      g/s             
    var_desc:   Model species PNO3                                           ...